In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 8.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c827b98162430ea833db480a01a0d4244f16910a55925bbb3706033b43bae487
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import random
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType, StructType
from pyspark.sql.functions import split
from pyspark.sql.functions import *
from pyspark.sql.functions import window

In [ ]:
spark = SparkSession\
.builder\
.appName('Project2')\
.master("local[*]")\
.config("spark.executor.memory", "70g")\
.config("spark.driver.memory", "50g")\
.config("spark.memory.offHeap.enabled",True)\
.config("spark.memory.offHeap.size","16g")\
.appName("sampleCodeForReference")\
.getOrCreate()

In [ ]:
project_id = 'big-query-test-350401'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://springboard2/


In [ ]:
bucket_name = 'springboard2'
!gsutil -m cp -r gs://{bucket_name} /content/drive/My\ Drive/ 

Copying gs://springboard2/full_dataset_000000000000.parquet...
Copying gs://springboard2/full_dataset_000000000001.parquet...
Copying gs://springboard2/full_dataset_000000000005.parquet...
Copying gs://springboard2/full_dataset_000000000002.parquet...
Copying gs://springboard2/full_dataset_000000000004.parquet...
Copying gs://springboard2/full_dataset_000000000006.parquet...
Copying gs://springboard2/full_dataset_000000000008.parquet...
Copying gs://springboard2/full_dataset_000000000007.parquet...
Copying gs://springboard2/full_dataset_000000000009.parquet...
Copying gs://springboard2/full_dataset_000000000003.parquet...
Copying gs://springboard2/full_dataset_000000000010.parquet...
Copying gs://springboard2/full_dataset_000000000011.parquet...
Copying gs://springboard2/full_dataset_000000000012.parquet...
Copying gs://springboard2/full_dataset_000000000013.parquet...
Copying gs://springboard2/full_dataset_000000000014.parquet...
Copying gs://springboard2/full_dataset_000000000015.par

In [ ]:
import os, json
import pandas as pd

path_to_json = '/content/drive/MyDrive/springboard_capstone2'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files)

['full_dataset_000000000000.json', 'full_dataset_000000000001.json', 'full_dataset_000000000002.json', 'full_dataset_000000000004.json', 'full_dataset_000000000009.json', 'full_dataset_000000000003.json', 'full_dataset_000000000005.json', 'full_dataset_000000000008.json', 'full_dataset_000000000006.json', 'full_dataset_000000000007.json', 'full_dataset_000000000010.json', 'full_dataset_000000000013.json', 'full_dataset_000000000011.json', 'full_dataset_000000000015.json', 'full_dataset_000000000014.json', 'full_dataset_000000000012.json', 'full_dataset_000000000016.json', 'full_dataset_000000000017.json', 'full_dataset_000000000018.json', 'full_dataset_000000000019.json', 'full_dataset_000000000020.json', 'full_dataset_000000000022.json', 'full_dataset_000000000026.json', 'full_dataset_000000000021.json', 'full_dataset_000000000023.json', 'full_dataset_000000000024.json', 'full_dataset_000000000025.json', 'full_dataset_000000000027.json', 'full_dataset_000000000028.json', 'full_dataset

In [ ]:
!pip install fsspec
!pip install gcsfs
import pandas as pd
df = pd.read_parquet('gs://springboard2/full_dataset_000000000002.parquet')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.7 MB/s 
     |████████████████████████████████| 271 kB 55.2 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 57.8 MB/s 


In [ ]:
df

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,NaN,2,1492658319,1492658319,20170419,"{'visits': 1, 'hits': 3, 'pageviews': 3.0, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Sou...","[{'index': 4, 'value': 'South America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 20, 'minu...",5948420746588309879,None,None,Organic Search,Not Socially Engaged
1,NaN,2,1479660647,1479660647,20161120,"{'visits': 1, 'hits': 15, 'pageviews': 9.0, 't...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",3429802729722245316,None,None,Organic Search,Not Socially Engaged
2,NaN,1,1486166963,1486166963,20170203,"{'visits': 1, 'hits': 24, 'pageviews': 17.0, '...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 16, 'minu...",2663293912628260778,None,None,Referral,Not Socially Engaged
3,NaN,1,1497245755,1497245755,20170611,"{'visits': 1, 'hits': 21, 'pageviews': 18.0, '...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Eastern...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu...",0461478766934342347,None,None,Organic Search,Not Socially Engaged
4,NaN,1,1472868721,1472868721,20160902,"{'visits': 1, 'hits': 9, 'pageviews': 7.0, 'ti...","{'referralPath': None, 'campaign': 'AW - Dynam...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 19, 'minu...",4985667982350539192,None,None,Paid Search,Not Socially Engaged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9052,NaN,1,1479152748,1479152748,20161114,"{'visits': 1, 'hits': 13, 'pageviews': 10.0, '...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': '(not set)', 'subContinent': '(n...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",0082404799374834117,None,None,Direct,Not Socially Engaged
9053,NaN,1,1472622436,1472622436,20160830,"{'visits': 1, 'hits': 13, 'pageviews': 9.0, 't...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu...",4923882521076397069,None,None,Direct,Not Socially Engaged
9054,NaN,1,1500477065,1500477065,20170719,"{'visits': 1, 'hits': 13, 'pageviews': 10.0, '...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",7786226692693598037,None,None,Organic Search,Not Socially Engaged
9055,NaN,1,1488856882,1488856882,20170306,"{'visits': 1, 'hits': 13, 'pageviews': 13.0, '...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 19, 'minu...",7579259479397862179,None,None,Organic Search,Not Socially Engaged
